# Create undersampled k-space
This demonstration shows how to create different undersampled k-space data which can be used either directly for image reconstruction or used to simulate MR data acquisition of a new object.


First version: 6th of March 2022 
Author: Christoph Kolbitsch 
Copyright 2015 - 2021 Physikalisch-Technische Bundesanstalt. 

This is software developed for the Collaborative Computational Project in Synergistic Reconstruction for Biomedical Imaging 
(http://www.ccpsynerbi.ac.uk/).

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [ ]:
#%% make sure figures appears inline and animations works
%matplotlib notebook

In [ ]:
__version__ = '0.1.1'

import numpy as np

# import engine module
import sirf.Gadgetron as mr

# import further modules
import os
from numpy.lib.stride_tricks import as_strided

%matplotlib inline
import matplotlib.pyplot as plt



# Utilities

In [ ]:
def plot_2d_t_image(vol, title, clims=None, cmap='viridis'):
    idim = vol.shape
    fig, ax = plt.subplots(1,3)
    fig.suptitle(title)
    if clims is None:
        clims = [vol.min(), vol.max()]
    ax[0].imshow(vol[idim[0]//2,:,:], cmap=cmap, clim=clims)
    ax[1].imshow(vol[:, idim[1]//2,:], cmap=cmap, clim=clims)
    ax[1].set_ylabel('Cardiac phases')
    ax[2].imshow(vol[:,:,idim[2]//2], cmap=cmap, clim=clims)
    ax[2].set_ylabel('Cardiac phases')
    
    for ind in range(3):
        ax[ind].set_xticks([])
        ax[ind].set_yticks([])

    

def crop_and_fill(templ_im, vol):
    """Crop volumetric image data and replace image content in template image object"""
    # Get size of template image and crop
    idim_orig = templ_im.as_array().shape
    idim = (1,)*(3-len(idim_orig)) + idim_orig
    offset = (np.array(vol.shape) - np.array(idim)) // 2
    vol = vol[offset[0]:offset[0]+idim[0], offset[1]:offset[1]+idim[1], offset[2]:offset[2]+idim[2]]
    
    # Make a copy of the template to ensure we do not overwrite it
    templ_im_out = templ_im.copy()
    
    # Fill image content 
    templ_im_out.fill(np.reshape(vol, idim_orig))
    return(templ_im_out)


'''
Variable density Cartesian sampling taken from
https://github.com/js3611/Deep-MRI-Reconstruction/blob/master/utils/compressed_sensing.py
'''

def normal_pdf(length, sensitivity):
    return np.exp(-sensitivity * (np.arange(length) - length / 2)**2)


def cartesian_mask(shape, acc, sample_n=10):
    """
    Sampling density estimated from implementation of kt FOCUSS
    shape: tuple - of form (..., nx, ny)
    acc: float - doesn't have to be integer 4, 8, etc..
    """
    N, Nx, Ny = int(np.prod(shape[:-2])), shape[-2], shape[-1]
    pdf_x = normal_pdf(Nx, 0.5/(Nx/10.)**2)
    lmda = Nx/(2.*acc)
    n_lines = int(Nx / acc)

    # add uniform distribution
    pdf_x += lmda * 1./Nx

    if sample_n:
        pdf_x[Nx//2-sample_n//2:Nx//2+sample_n//2] = 0
        pdf_x /= np.sum(pdf_x)
        n_lines -= sample_n

    mask = np.zeros((N, Nx))
    for i in range(N):
        idx = np.random.choice(Nx, n_lines, False, pdf_x)
        mask[i, idx] = 1

    if sample_n:
        mask[:, Nx//2-sample_n//2:Nx//2+sample_n//2] = 1

    size = mask.itemsize
    mask = as_strided(mask, (N, Nx, Ny), (size * Nx, size, 0))

    mask = mask.reshape(shape)

    return mask


## (A) Fully sampled k-space data

Load in fully sampled k-space data and preprocess it.

In [ ]:
# Load MR AcquisitionData
mr_acq = mr.AcquisitionData('/home/jovyan/tmp/cine_64_32ph.h5')
preprocessed_data = mr.preprocess_acquisition_data(mr_acq)

In [ ]:
# Calculate image
recon = mr.FullySampledReconstructor()
recon.set_input(preprocessed_data)
recon.process()
im_mr = recon.get_output()

In [ ]:
# Display it
plot_2d_t_image(np.abs(im_mr.as_array()), 'Original image', cmap="Greys_r")


## (B) Create undersampling mask

The acquisitions for all cardiac phases are stored as one big vector, so we get the phase encoding index $ky$ and cardiac phase index $cph$ for all acquisitions. 

In [ ]:
ky_index = preprocessed_data.get_ISMRMRD_info('kspace_encode_step_1')
cph_index = preprocessed_data.get_ISMRMRD_info('phase')

Calculate number of phase encoding steps and cardiac phases

In [ ]:
ky_num = int(np.max(ky_index)+1)
cph_num = int(np.max(cph_index)+1)
print(f'Nky {ky_num} - Ncph {cph_num}')

Create and visualise sampling mask for all phases with a total undersampling factor $R$ and a fully sampled centre of width $F$

In [ ]:
R = 4
F = int(ky_num/10)
msk = cartesian_mask([cph_num, ky_num, 1], R, sample_n=F)

fig, ax = plt.subplots(1,1)
ax.imshow(msk[:,:,0])
ax.set_xlabel('$k_y$')
ax.set_ylabel('Cardiac phase');

## (C) Create undersampled data

Now we know which k-space points to select, we need to select them and create a new `AcquisitionData` object. We will go through all cardiac phases and select the corresponding $ky$ indices. If the heartrate changes, the RR-cycle varies in length and hence certain $ky$ indices cannot be acquired. This is usually compensated for by interpolation. Here we are simply ignoring these missing values.

In [ ]:
acq_us = preprocessed_data.new_acquisition_data(empty=True)

# Create raw data
for cnd in range(cph_num):
    for ynd in range(ky_num):
        if msk[cnd, ynd, 0] == 1:
            cidx = np.where((ky_index == ynd) & (cph_index == cnd))[0]
            if len(cidx) > 0:
                cacq = preprocessed_data.acquisition(cidx)
                acq_us.append_acquisition(cacq)
            else:
                print(f'ky {ynd} - cph {cnd} not found')

acq_us.sort()     

## (D) Simple reconstruction of the undersampled phantom

Now we will do a simple reconstruction by defining and `AcquisitionModel` based on the `AcquisitionData` and then call `backward()` (i.e. Fourier transform).

In [ ]:
# Original data
csm_orig = mr.CoilSensitivityData()
csm_orig.smoothness = 200
csm_orig.calculate(preprocessed_data)

A_orig = mr.AcquisitionModel(preprocessed_data, im_mr)
A_orig.set_coil_sensitivity_maps(csm_orig)

im_orig = A_orig.backward(preprocessed_data)


# Undersampled data
csm_us = mr.CoilSensitivityData()
csm_us.smoothness = 200
csm_us.calculate(acq_us)

A_us = mr.AcquisitionModel(acq_us, im_mr)
A_us.set_coil_sensitivity_maps(csm_us)

im_us = A_us.backward(acq_us)

In [ ]:
# Display it
plot_2d_t_image(np.abs(im_orig.as_array()), 'Original image', cmap="Greys_r")
plot_2d_t_image(np.abs(im_us.as_array()), 'Undersampled image', cmap="Greys_r")

## (E) Simulate new data

Get image from scipy

In [ ]:
import scipy.misc
face = scipy.misc.face()
scale_fac = face.shape[0]//im_us.as_array().shape[1]
face = face[::scale_fac,::scale_fac,:]
face_grey = 0.2125*face[:,:,0] + 0.7154*face[:,:,1] + 0.0721*face[:,:,2]

Ensure it is the same size as the original image

In [ ]:
idim = im_us.shape
face_grey = face_grey[:idim[1], :idim[2]]

The image is only a single frame so we have to make copies for the differen cardiac phases

In [ ]:
face_grey = np.tile(face_grey[np.newaxis,:,:], [cph_num, 1, 1])

In order to be able to pass this image on to the `AcquisitionModel` we need an object of type `ImageData`. The easiest way to achieve this is to make a copy of an already exisiting `ImageData` object and fill it with the new content

In [ ]:
im_new = im_us.copy()
im_new.fill(face_grey)

Now we can simulate a data acquisition and carry out simple reconstruction

In [ ]:
# Create k-space data
acq_us_new = A_us.forward(im_new)

# Simple reconstruction
im_us_new = A_us.backward(acq_us_new)

# Display it
plot_2d_t_image(np.abs(im_new.as_array()), 'New original image', cmap="Greys_r")
plot_2d_t_image(np.abs(im_us_new.as_array()), 'New undersampled image', cmap="Greys_r")